This is Markdown cell with explanation, how I did the rest.
1. I used Microsoft Excel App/Data/from WEB, in the Navigator box I selected the table for import and saved a file as CSV.
2. Using 'Find and add data' button in Jupyter notebook, I drugged needed CSV file in files and picked 'Insert pandas DataFrame'.
    After you do that Jupyter Notebook automaticaly creates first block of code with hidden credentials.
3. Changed default data frame name to Toronto
4. Dropped rows containing value 'Not assigned' in Borough column
5. Replaced 'Not assigned' values in column Neighbourhood by column Borough values
6. Grouped by Postcode and reindexed Data Frame
7. Result of shape is (103,3)

In [1]:
# The code was removed by Watson Studio for sharing.

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [ ]:
Toronto = pd.read_csv(body) 
# 3. Changed default data frame name to Toronto
Toronto.head()

In [2]:
Toronto.drop(Toronto.loc[Toronto['Borough']=='Not assigned'].index, inplace=True) 
# 4. Dropped rows containing value 'Not assigned' in Borough column
Toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [3]:
Toronto['Neighbourhood'].replace('Not assigned', Toronto['Borough'], inplace=True)
# 5. Replaced 'Not assigned' values in column Neighbourhood by column Borough values
Toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [4]:
Toronto=Toronto.groupby("Postcode").agg(lambda x: ', '.join(set(x))).reset_index()
# 6. Grouped by Postcode and reindexed Data Frame
Toronto.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
Toronto.shape

(103, 3)

This is Markdown cell with explanation of my further actions. 
1. Using 'Find and add data' button in Jupyter notebook, I drugged Geospatial_Coordinates.csv file in files and picked 'Insert pandas DataFrame'. This time credentials were not hidden, so added # @hidden_cell option
2. Changed default name to Coord and renamed first column to Postcode so it has the same name as column in Toronto DF.
3. Created new DF TorCoor by merging two DFs in one.

In [6]:
# The code was removed by Watson Studio for sharing.

In [7]:
# 2. Changed default name to Coord and renamed first column to Postcode so it has the 
#same name as column in Toronto DF.
Coord = pd.read_csv(body)
Coord.columns = ['Postcode', 'Latitude','Longitude']
Coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
# 3. Created new DF TorCoor by merging two DFs in one.
TorCoor=pd.merge(Toronto, Coord, on=['Postcode'])
TorCoor.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
